 <center> Module EL 3019 
 
 <font color="#0066CC"> **Méthode des K plus proches voisins (Régression)** </font> </center>
<hr>

1. Objectifs  
2. Chargement des bibliothèques et du jeu de données 
3. Prérequis
4. KPPV en régression

<hr>

<font color="#0066CC"> 1. Les objectifs  </font> 
- Ce TP a pour objectif d'apprendre à réaliser le méthode des k plus proches voisins en régression à l'aide de python et des packages appropriés
- En introduction, nous utiliserons cette méthode sur le jeux de données complet puis nous réaliserons un échantillonage dans l'objectif de déterminer le k optimal.

<u>*remarque importante*</u> : les statistiques univariées (disribution, histogrammes) et bivariées (correlations entre les variables, scatter plot) sont intentionnellement omises dans ce TP  (ce n'est pas son objectif). Il n'en demeure pas moins qu'elles doivent être **systématiquement** réalisées lors de l'étude d'un jeux de données 
<br>

<font color="#0066CC">2. Chargement des bibliothèques et du jeu de données </font>    

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

# Importation des données

file = "SURF_data.xlsx"
data = pd.read_excel(<COMPLETER ICI> , decimal = ',')
data.info()


<font color="#0066CC"> 3. Pre-requis : les étapes </font>  
Le Jeu de données brut comprend 10 variables. Notre objectif est d'évaluer la méthode des kppv pour l'estimation de la variable *chauff*. La variable *refroid* ne nous interessa pas et sera eliminée. 
Nous avons donc   
* un data.frame X (Predicteur) composé des variables prédictives suivantes :  
  - compacité
  - Surface
  - mur
  - toit
  - hauteur
  - orientation
  - vitrage
  - or_vitre
* un dataframe Y composé uniquement de la variable *chauff*  

<br>

 
3.1. Nous verifions dans un premier temps l'absence / présence de données manquantes   

3.2  Certaines variables, bien que numériques, ne sont pas formatées correctement. En effet, les variables orientation et or_vit se référent à un orientation de type ; nord, sud, est, ouest,.... Il convient donc de les transformer. Deux étapes sont necessaires : (i) la labellisation puis(ii) un One Hot encoding

3.3 La méthode des KPPV est fondée sur la calcul des distances entre les différents individus (lignes du prédicteur). Il est donc important de standardiser les données de type float64






In [ ]:
#-> on vérifie la présence / absence de données manquantes
data.isna().sum()
#-> on élimine la variable refroid
data.drop('refroid', axis = 1, inplace=True)
data.head()

Vous devez donc

1. marquer les variables catéorielles (méthode `.astype`)
2. sélectionner les variables catégorielles (méthode `.select_dtypes(include = ...)`)
3. utiliser la librairie `OneHotEncoder` pour transformer les variables
4. recréer un dataframe (la sortie de OHE est un array)
5. sélectionner les variables quantitatives (méthode `.select_dtypes(exclude = ...)`)
6. utiliser la librairie `StandardScaler` pour transformer les variables
7. recréer un dataframe (la sortie de SC est un array)
8. concaténer les deux dataframes

Enfin, créer les matrices X (variables explicatives) et y (réponse)

Les cellules suivantes vous guident dans la démarche standard qui permet d'utiliser les classes de *preprocessing*, en identifiant le stransformations que vous pourriez ensuite utiliser sur de nouveles données. Plus bas, on vous livre une version plus rapide mais moins précautionneuse. 

In [ ]:
#___________________________________________________________
#  TRANSFORMATION DES VARIABLES : ONE HOT ENCODING
#___________________________________________________________
# 1. tranformation en variables catégorielle des deux variables
data[['orientation','or_vitre']] = data[['orientation','or_vitre']].astype('category')
# 2. One hot encoding
#---------------------------------------------------
# 2.1. On selectionne les variables catégorielles
df_cat   = data.select_dtypes(include = <COMPLETER ICI>)
name_cat = df_cat.columns
#---------------------------------------------------
#2.2 Appel et utilisation de la librairie pour l'encodage
OH     = OneHotEncoder(handle_unknown='error',sparse=False)
fit_OH = OH.fit(<COMPLETER ICI>)
fit_data_OH    = fit_OH.transform(<COMPLETER ICI>)
column_name_OH = fit_OH.get_feature_names(<COMPLETER ICI>)
#---------------------------------------------------
#2.3 On recrée un dataframe contenant les variables OneHot
dfw_OH = pd.DataFrame(fit_data_OH,columns = column_name_OH)
dfw_OH.reset_index(drop=True, inplace=True)

In [ ]:
#___________________________________________________________
#  STANDARDISATION
#___________________________________________________________
# 1. sélection des variables numériques 
dfw_num   = data.select_dtypes(exclude = <COMPLETER ICI>)

#----------------------------------------------------------
# 2. Centrage et réduction  
SC       =  StandardScaler()
fit_SC   =  SC.fit_transform(<COMPLETER ICI>)
dfw_SC   =  pd.DataFrame(fit_SC, columns = dfw_num.columns)
dfw_SC.reset_index(drop=True, inplace=True)

In [ ]:
#____________________________________________________________
# DATAFRAME
#---------------------------------------------------------
# Dataframe contenant les variables standardisées / One Hot encodées
#  On crée différents dataframe
#---------------------------------------------------------
# 1. Dataframe contenant Toutes les variables
dfw_All = pd.concat(<COMPLETER ICI>, axis = <COMPLETER ICI>)
dfw_All

#---------------------------------------------------------
# 2. Le dataframe contenant les variables prédictives (Predicteurs)
X_num = dfw_All.drop('chauff', axis = 1)

#---------------------------------------------------------
# 2. Le dataframe contenant la variables à prédire (en régression)
Y_num = dfw_All['chauff']

**Une version rapide alternative..**

In [ ]:

var_quanti = ['compacité', 'Surface', 'mur', 'toit', 'hauteur', 'vitrage', 'chauff']
data[var_quanti] = StandardScaler().fit_transform(data[var_quanti])
#
var_quali = ['orientation','or_vitre']
data_quali_encoded = pd.get_dummies(data[var_quali])
# 
df = pd.concat([data[var_quanti], data_quali_encoded], axis = 1)
#
X_num = df.drop('chauff', axis=1)
Y_num = df['chauff']

<font color="#0066CC"> **4. KPPV en régression** </font>     
<font color="#0066CC"> 4.1 Régression avec  k fixe  </font>
* Estimation des valeurs  
Dans le cadre de la régression, l'objectif est de trouver le nombre de plus proche voisins (k) qui permet de "s'approcher" au mieux des valeurs observées dela variable *chauff*.   
Comme nous l'avons vu dans le cours, Deux étapes sont nécessaires :  
  * La phase d'entrainement ou l'on effectue le paramètrage (on fixe le k) sur un échantillon du jeu de données (70%) 
  * Puis la phase de validation (30%) où l'on évalue si le k fixé en entrainement estime correctement les valeurs observées (annotées) sur un autre échantillon du jeux de données (cf. shéma 1)  
  * par défaut, nous fixons le nombre de k plus proches voisins = 20


In [ ]:
#_______________________________________________________________________________
# PARTITION DU JEUX DE DONNEES (70% TRAIN - 30% VALIDATION)
X_train, X_test, Y_train, Y_test = train_test_split(<COMPLETER ICI>, test_size= 0.30)

#______________________________________________________________________________
# KNN EN REGRESSION
#-> on récupére le nombre de valeurs en test 
n_test = len(Y_test)
KNN_Reg = KNeighborsRegressor(<COMPLETER ICI>)
KNN_Reg.fit(<COMPLETER ICI>)
Y_pred = KNN_Reg.predict(<COMPLETER ICI>)
#_____________________________________________________________________________
# On récupère les valeurs prédites
#Y_pred


* Qualité de l'estimation 
 La qualité  de l'estimation correspond à la moyenne de la somme des carrés des écarts entre les valeurs observées et les valeurs prédites (appelée erreur quadratique moyenne ou MSE en anglais) . Plus cette valeur est faible, meilleure sera la qualité de l'ajustement. 

In [ ]:
MSE = <COMPLETER ICI>
MSE

Visualisons la relation à l'aide d'un scatter plot entre les valeurs observée et prédites

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
df_result = pd.DataFrame({'obs' : Y_test, 'pred': Y_pred})
_ = sns.scatterplot(<COMPLETER ICI>)

<font color="#0066CC"> 4.2 Régression avec différentes valeurs de k </font>
* Jusqu'à présent, nous avons fixé le nombre de plus proches voisins (=20) et nous obtenons une valeur de MSE qui reflète une certaine qualité de l'ajustement. La question que l'on peut se poser est alors la suivante :  Peut on améliorer la qualité de l'ajustement en modifiant le nombre de k ?

* Pour y répondre, nous allons incrémenter le nombre de k et calculer, pour chaque incrément de k, l'erreur Quadratique moyenne. 

In [ ]:
kmax = 80
#---> On stocke dans un array les erreurs quadratiques moyennes
error_Reg =  np.zeros(kmax)
#--> On fait varier les valeurs de k = 1 à kmax par pas de 1
kppv = np.arange(1,kmax+1,1)
#--> boucle 
for k in kppv:
    <COMPLETER ICI>
    Y_pred = <COMPLETER ICI>
    #--> calcul MSE
    MSE = <COMPLETER ICI>
    #--> et stockage dans la liste
    error_Reg[k-1] = MSE
   

* Nous construisons le graphique MSE = f(k) et nous calculons le k optimal


In [ ]:
df_result_Reg = pd.DataFrame({ 
                          'k'   : kppv,
                          'MSE' : error_Reg
                        })

k_opt = df_result_Reg['MSE'].values.argmin(0) + 1

_ = df_result_Reg.plot(x='k', y='MSE', style='.-')


* Nous comparons les valeurs observées (en x)  par rapport aux valeurs prédites pour k = 15 et pour le k optimal (en y). En théorie, si les prédictions correspondent "exactement" aux observations, on observe une droite. Comme ce n'est jamais le cas, plus les points se 'rapprocheront' d'une droite meilleure sera la prédiction

In [ ]:

#-> On calcule les valeurs pour la k optimal
KNN_Reg = <COMPLETER ICI>
KNN_Reg.fit(<COMPLETER ICI>)
Y_pred_opt = KNN_Reg.<COMPLETER ICI>

#--> On ajoute au dataFrame la variable pred_opt = résultats obtenus avec le k optimal
df_result['pred_opt'] = Y_pred_opt

_ = sns.scatterplot(x='obs', y='pred', data=df_result, label='k=20')
_ = sns.scatterplot(x='obs', y='pred_opt', data=df_result, label='k=kopt')


In [ ]:
ax = df_result.sort_values(by='obs').plot(x='obs', y='pred', style='.-')
df_result.sort_values(by='obs').plot(x='obs', y='pred_opt', style='.-', ax=ax)
_ = ax.plot([-1.5, 2],[-1.5, 2], label='optimal')

* Comme on peut l'observer, il y a moins de dispersion des points lorsque nous utilisons le k_optimal (en marron sur le graphique)
